In [28]:
import torch
import torchvision
from torchvision import transforms, datasets
from torch import nn, optim
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt

In [29]:
model = torchvision.models.resnet18(pretrained=True)

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)


In [30]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [31]:
# removing checkpoints, bugging
!rm -rf /content/data/train/.ipynb_checkpoints
!rm -rf /content/data/val/.ipynb_checkpoints


In [32]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

train_data = datasets.ImageFolder('/content/data/train', transform=transform)
val_data = datasets.ImageFolder('/content/data/val', transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

print(f"✅ Loaded {len(train_data)} training images and {len(val_data)} validation images.")


✅ Loaded 171 training images and 223 validation images.


In [33]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [35]:
epochs = 10
for epoch in range(epochs):
    model.train()  #
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in train_loader:
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}, Accuracy: {100 * correct / total}%")


Epoch 1/10, Loss: 0.3447714665283759, Accuracy: 91.2280701754386%
Epoch 2/10, Loss: 0.2944112842281659, Accuracy: 95.32163742690058%
Epoch 3/10, Loss: 0.1692212506507834, Accuracy: 92.98245614035088%
Epoch 4/10, Loss: 0.15554529055953026, Accuracy: 95.32163742690058%
Epoch 5/10, Loss: 0.12380111093322436, Accuracy: 95.90643274853801%
Epoch 6/10, Loss: 0.09528650219241779, Accuracy: 96.49122807017544%
Epoch 7/10, Loss: 0.11374134135742982, Accuracy: 97.6608187134503%
Epoch 8/10, Loss: 0.0922712329775095, Accuracy: 97.6608187134503%
Epoch 9/10, Loss: 0.2548397894327839, Accuracy: 94.15204678362574%
Epoch 10/10, Loss: 0.12430937153597672, Accuracy: 96.49122807017544%


In [36]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Validation Accuracy: {100 * correct / total}%")

Validation Accuracy: 86.99551569506727%


In [43]:
from PIL import Image

img = Image.open('/content/pic2.jpg')

img_tensor = transform(img).unsqueeze(0)

model.eval()
with torch.no_grad():
    output = model(img_tensor)
    _, predicted = torch.max(output.data, 1)

if predicted.item() == 0:
    print("Bus detected!")
else:
    print("Not a bus!")


Not a bus!
